<a href="https://colab.research.google.com/github/YongWanJin/KDTteam5_semiproject/blob/main/%EC%A0%84%EC%B2%98%EB%A6%AC_%EC%A2%8C%ED%91%9C%EC%88%98%EC%A0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 도움을 준 사이트 : https://wooiljeong.github.io/python/transform_coord/

### 좌표계 변환에 필요한 패키지 pyproj 설치
!pip install pyproj

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.3 MB 6.7 MB/s 


In [4]:
### 로컬 폴더에서 colab으로 파일 불러오기
# 사용한 데이터 : 서울시 우리마을가게 상권분석서비스(상권영역).csv

from google.colab import files
uploaded = files.upload()

Saving 서울시 우리마을가게 상권분석서비스(상권영역).csv to 서울시 우리마을가게 상권분석서비스(상권영역).csv


In [7]:
### colab으로 불러온 파일을 데이터프레임df에 저장 & 필요한 모듈 불러오기

import pandas as pd
import numpy as np
import pyproj
import folium
import io

df = pd.read_csv(io.StringIO(uploaded['서울시 우리마을가게 상권분석서비스(상권영역).csv'].decode('CP949')))
print(df)
print(df.info()) # '엑스좌표_값'과 '와이좌표_값'이 이미 int형으로 되어있음, 따로 전처리해줄 필요 없음

      기준_년월_코드 상권_구분_코드 상권_구분_코드_명    상권_코드       상권_코드_명  엑스좌표_값  와이좌표_값  \
0       202112        A       골목상권  2110753   시흥동 은행나무사거리  191934  438813   
1       202112        A       골목상권  2110427    노원 을지대학교병원  206265  459431   
2       202112        A       골목상권  2110755      동일여자고등학교  192300  439070   
3       202112        A       골목상권  2110739    시흥3동 우편취급국  191289  438014   
4       202112        A       골목상권  2111017      송파나루역 3번  210222  445516   
...        ...      ...        ...      ...           ...     ...     ...   
1666    202112        U       관광특구  1001493  동대문패션타운 관광특구  200974  451975   
1667    202112        U       관광특구  1001494    종로?청계 관광특구  199768  452273   
1668    202112        U       관광특구  1001495       잠실 관광특구  210191  446339   
1669    202112        U       관광특구  1001496   강남 마이스 관광특구  205310  445727   
1670    202112        U       관광특구  1001491      이태원 관광특구  199503  448326   

      시군구_코드    행정동_코드  형태정보  
0      11545  11545710   NaN  
1      11350 

In [9]:
### 엑스좌표값, 와이좌표값만 추출

coord_x = df[['엑스좌표_값']]
coord_y = df[['와이좌표_값']]
coord = pd.concat([coord_x, coord_y], axis=1)

print(coord)

      엑스좌표_값  와이좌표_값
0     191934  438813
1     206265  459431
2     192300  439070
3     191289  438014
4     210222  445516
...      ...     ...
1666  200974  451975
1667  199768  452273
1668  210191  446339
1669  205310  445727
1670  199503  448326

[1671 rows x 2 columns]


In [13]:
### 원본의 좌표계를 시각화 가능한 좌표계로 변환하기 위한 함수 정의

def coord_transfer(coord, p1_type, p2_type) :

  #coord: x, y 좌표 정보가 담긴 NumPy Array
  #p1_type: 입력 좌표계 정보 ex) 원본 자료의 좌표계 종류는 epsg:5181 라고 함.
  #p2_type: 출력 좌표계 정보 ex) 시각화할때 쓰는 좌표게 종류는 epsg:4326

  p1 = pyproj.Proj(init=p1_type)
  p2 = pyproj.Proj(init=p2_type)
  fx, fy = pyproj.transform(p1, p2, coord[:, 0], coord[:, 1])

  return np.dstack([fx, fy])[0]


In [14]:
### 좌표계 변환 실시

coord = np.array(coord) # 함수에 적용하기 위해 데이터프레임'coord'의 타입을 np.array로 변환
p1_type = 'epsg:5181'
p2_type = 'epsg:4326'

result = coord_transfer(coord, p1_type, p2_type)
print(result)

[[126.90884426  37.44868725]
 [127.07097812  37.63446919]
 [126.91297783  37.45100597]
 ...
 [127.11527495  37.51647607]
 [127.0600594   37.51100291]
 [126.99437686  37.53443511]]


/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  # This is added ba

In [15]:
### 변환된 좌표계를 원본 데이터 df에 추가

df['경도'] = result[:,0]
df['위도'] = result[:,1]
print(df)

      기준_년월_코드 상권_구분_코드 상권_구분_코드_명    상권_코드       상권_코드_명  엑스좌표_값  와이좌표_값  \
0       202112        A       골목상권  2110753   시흥동 은행나무사거리  191934  438813   
1       202112        A       골목상권  2110427    노원 을지대학교병원  206265  459431   
2       202112        A       골목상권  2110755      동일여자고등학교  192300  439070   
3       202112        A       골목상권  2110739    시흥3동 우편취급국  191289  438014   
4       202112        A       골목상권  2111017      송파나루역 3번  210222  445516   
...        ...      ...        ...      ...           ...     ...     ...   
1666    202112        U       관광특구  1001493  동대문패션타운 관광특구  200974  451975   
1667    202112        U       관광특구  1001494    종로?청계 관광특구  199768  452273   
1668    202112        U       관광특구  1001495       잠실 관광특구  210191  446339   
1669    202112        U       관광특구  1001496   강남 마이스 관광특구  205310  445727   
1670    202112        U       관광특구  1001491      이태원 관광특구  199503  448326   

      시군구_코드    행정동_코드  형태정보          경도         위도  
0      11545  1154571

In [ ]:
### 잘 되나 테스트

sample = df.sample(n=10) # 임의로 10개 뽑아봄
lat_c, lon_c = 37.53165351203043, 126.9974246490573
m = folium.Map(location=[lat_c, lon_c], zoom_start=6)
for _, row in sample.iterrows():
    lat, lon = row['위도'], row['경도']
    folium.Marker(location=[lat, lon]).add_to(m)
m

In [17]:
### colab에 저장, 다운로드 할수 있는 상태가 됨.

df.to_csv('directory\상권영역_위도경도추가.csv')

# 이제 이 데이터를 가지고 시각화를 할 수 있게됨.